In [1]:
from collections import defaultdict, Counter
from itertools import combinations
import random
import json
from string import punctuation

import numpy as np
import pandas as pd

import usaddress
from tqdm import tqdm
import networkx as nx
import cleanco

In [2]:
# df_2020 = pd.read_excel('2015-2020 Evictions Maricopa County.xlsx', sheet_name='2020')
df_2020 = pd.read_csv('2015-2020 Evictions Maricopa County - 2020.csv')
df_2020.head(30)

/Users/yixiang/anaconda3/envs/pper/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (10,18) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,File Date,Case Number,SubCategory,Immediate,Court,Case Status,Case Status Date,Monetary Judgment,Judgment For,Address,...,Def2 Ph,Plaintiff Name,Plaintiff1 Address,Plaint1 Zip,Plaintiff1 Email,Plaintiff1 Ph,Plt FTA?,Plaintiff2 Name,Plaintiff2 Email,Plaintiff2 Ph
0,1/2/20,CC2020000490000,Poss Prop,N,Agua Fria,Adjudicated,1/10/20,"$3,653.95",Plaintiff,"10751 W. DAVIS LANE, AVONDALE",...,NaN,LSF9 MASTER PARTICIPATION TRUST,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN
1,1/2/20,CC2020000495000,Poss Prop,N,Kyrene,Adjudicated,1/9/20,"$4,414.61",Plaintiff,"5102 E PIEDMONT ROAD 2244, PHOENIX",...,NaN,FPA/WC PINNACLE PHASE I LLC,"5151 E GUADALUPE ROAD, PHOENIX AZ",85044,NaN,(480) 496-6161,N,NaN,NaN,NaN
2,1/2/20,CC2020000496000,Poss Prop,N,San Marcos,Adjudicated,1/9/20,"$6,098.23",Plaintiff,"3800 W CHANDLER BLVD 1297, CHANDLER",...,NaN,GS CHANDLER LLC,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN
3,1/2/20,CC2020000497000,Residential,N,San Marcos,Adjudicated,1/9/20,"$2,823.59",Plaintiff,"2800 N ARIZONA AVE 234, CHANDLER",...,NaN,DESERT JEWEL APARTMENTS LLC,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN
4,1/2/20,CC2020000498000,Rent,N,University Lakes,Adjudicated,1/9/20,"$5,578.58",Plaintiff,"1720 E Broadway rd 2127, Tempe",...,NaN,Marbeya,"4222 E Thomas Rd #230, Phoenix AZ",85018,NaN,NaN,N,NaN,NaN,NaN
5,1/2/20,CC2020000500000,Poss Prop,N,San Marcos,Adjudicated,1/9/20,"$5,712.99",Plaintiff,"3950 W CHANDLER BLVD 2072, CHANDLER",...,NaN,RPC CHAMDLER APARTMENTS LLC,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN
6,1/2/20,CC2020000501000,Poss Prop,N,Maryvale,Adjudicated,1/10/20,"$1,604.50",Plaintiff,"6301 N 64th Dr 11, Phoenix",...,NaN,ACKK Enterprises LLC,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN
7,1/2/20,CC2020000502000,Rent,N,Country Meadows,Adjudicated,1/8/20,"$3,274.59",Plaintiff,"12175 W MCDOWELL RD 1308, AVONDALE",...,NaN,ASHTON POINTE HOLDINGS LLC,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN
8,1/2/20,CC2020000503000,Poss Prop,N,Maryvale,Adjudicated,1/6/20,NaN,NaN,"4030 N 44th Ave 2077, Phoenix",...,NaN,Cove Phoenix LLC,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN
9,1/2/20,CC2020000508000,Rent,N,Arrowhead,Adjudicated,1/9/20,"$3,961.02",Plaintiff,"8092 W PARADISE LN 1140, PEORIA",...,NaN,ENCLAVE INVESTORS LP,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN


In [4]:
df_2020.shape[0]

37760

In [5]:
df_2020.columns

Index(['File Date', 'Case Number', 'SubCategory', 'Immediate', 'Court',
       'Case Status', 'Case Status Date', 'Monetary Judgment', 'Judgment For',
       'Address', 'Zip', 'Writ Date', 'Plt Legal Rep?', 'Def Legal Rep?',
       'Claim Amount', 'Bench Trial?', 'Defendant1 Name', 'Defendant1 Address',
       'Def1 Zip', 'Def1 Email', 'Deft1 Ph', 'Def FTA?', 'Defendant2 Name',
       'Def2 Email', 'Def2 Ph', 'Plaintiff Name', 'Plaintiff1 Address',
       'Plaint1 Zip', 'Plaintiff1 Email', 'Plaintiff1 Ph', 'Plt FTA?',
       'Plaintiff2 Name', 'Plaintiff2 Email', 'Plaintiff2 Ph'],
      dtype='object')

In [252]:
df_2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37760 entries, 0 to 37759
Data columns (total 34 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   File Date           37760 non-null  object
 1   Case Number         37760 non-null  object
 2   SubCategory         37760 non-null  object
 3   Immediate           37760 non-null  object
 4   Court               37760 non-null  object
 5   Case Status         37760 non-null  object
 6   Case Status Date    37760 non-null  object
 7   Monetary Judgment   25197 non-null  object
 8   Judgment For        25387 non-null  object
 9   Address             34355 non-null  object
 10  Zip                 34355 non-null  object
 11  Writ Date           11095 non-null  object
 12  Plt Legal Rep?      37760 non-null  object
 13  Def Legal Rep?      37760 non-null  object
 14  Claim Amount        37760 non-null  object
 15  Bench Trial?        37760 non-null  object
 16  Defendant1 Name     37

In [253]:
df_2020.describe()

,File Date,Case Number,SubCategory,Immediate,Court,Case Status,Case Status Date,Monetary Judgment,Judgment For,Address,...,Def2 Ph,Plaintiff Name,Plaintiff1 Address,Plaint1 Zip,Plaintiff1 Email,Plaintiff1 Ph,Plt FTA?,Plaintiff2 Name,Plaintiff2 Email,Plaintiff2 Ph
count,37760,37760,37760,37760,37760,37760,37760,25197,25387,34355,...,187,37760,14709,14689,203,2653,37760,1700,24,95
unique,253,37760,6,2,26,2,303,21951,2,32347,...,187,12459,4966,353,178,711,2,1036,24,70
top,1/17/20,CC2020029497000,Poss Prop,N,Country Meadows,Adjudicated,2/25/20,$0.00,Plaintiff,"7007 W INDIAN SCHOOL RD 01-1287, PHOENIX",...,480-562-7173,DEL MAR TERRACE APTS LLC,"4222 E Thomas Rd #230, Phoenix AZ",85018,office@scottclarklaw.com,(602) 957-7877,N,SATYA VERMA,ysabel.escarcega@gmail.com,(602) 242-0237
freq,778,1,31816,37305,3056,37488,789,233,25305,7,...,1,247,485,2228,7,318,37757,64,1,7


In [247]:
df_2020.groupby(['Court'])['Court'].count().sort_values(ascending=False)

Court
Country Meadows      3056
Manistee             2924
Moon Valley          2482
Arcadia Biltmore     2355
Maryvale             2214
Kyrene               2096
Encanto              2052
Dreamy Draw          1787
West Mesa            1701
San Marcos           1673
West McDowell        1526
University Lakes     1412
North Mesa           1356
South Mountain       1348
East Mesa            1316
Desert Ridge         1166
North Valley         1141
McDowell Mountain    1036
Downtown              963
White Tank            820
Highland              748
Hassayampa            720
San Tan               655
Agua Fria             638
Arrowhead             559
Ironwood               16
Name: Court, dtype: int64

## Defendants

Related attributes: 'Defendant1 Name', 'Defendant1 Address', 'Def1 Zip', 'Def1 Email', 'Deft1 Ph', 'Def FTA?', 'Defendant2 Name', 'Def2 Email', 'Def2 Ph'

In [7]:
defendants1 = df_2020[['Defendant1 Name']].dropna()['Defendant1 Name'].tolist()
defendants2 = df_2020[['Defendant2 Name']].dropna()['Defendant2 Name'].tolist()
len(defendants1), len(defendants2), len(defendants1 + defendants2)

(37753, 12185, 49938)

In [122]:
dedup_defendants1 = set(map(lambda x: x.lower(), defendants1))
dedup_defendants2 = set(map(lambda x: x.lower(), defendants2))
dedup_all_defendants = dedup_defendants1 | dedup_defendants2
len(dedup_defendants1), len(dedup_defendants2), len(dedup_all_defendants)

(30230, 10688, 38590)

In [240]:
len(defendants1) - len(dedup_defendants1), len(defendants2) - len(dedup_defendants2), len(defendants1 + defendants2) - len(dedup_all_defendants)

(7523, 1497, 11348)

In [131]:
# import nltk
# from nltk.tag.stanford import NERTagger
# st = NERTagger('stanford-ner/all.3class.distsim.crf.ser.gz', 'stanford-ner/stanford-ner.jar')

# def is_person_name(text):
#     for sent in nltk.sent_tokenize(text):
#         tokens = nltk.tokenize.word_tokenize(sent)
#         tags = st.tag(tokens)
#         for tag in tags:
#             if tag[1]=='PERSON': print tag

In [444]:
# name detection
# https://github.com/philipperemy/name-dataset

In [137]:
# names with punctuations
for d in dedup_all_defendants:
    if any(p in d for p in punctuation):
        print(d)

marquetta garrett-mitchell
zi-yah thompson
traci williams-medley
catherine o'connor
jo-anne cassity
caitlyn metcalf-boat
melanie nez-armstrong
samantha rojas-guevara
irma nunez-espinoza
khamari grady-cloud
karl adam-narvesen
jacklynn d'agay
sabrina congdon-lervold
james o'kelley
rekee-terasheena walker mcgary
tenae griffin-reason
de'on roberts
r.tudor investments llc
sumer al-fayez
m. whitehead
marion vasquez-sanchez
nevyn ivers-frank
de'shon allen
w. billy king
bunetta gillespie-randles
michael stout-squires
daniel valencia-cervantes
racquel winkler-smith
taylor hall-brown
kelly byers-gustafson
petrocelli ciccione-jimenez
alexandra wester-blanco
yolanda ray-alexander
jeanne c'de baca
jose ochoa- carrera
anna-ellen graham-begay
ting-hsuan kuo
rosalind ellis- mcghee
ta'lor corely
laurie-lynn soares
takia-denise stewart
katelynn krpata-caron
yerania chavez-ruiz
melissa oviedo-owens
dr. robbins bibbs
krista scheinost-greening
j.r. benton
monique battee-freeman
danny marty-rodriguez
shelly

In [254]:
used_punctuations = defaultdict(int)
for d in dedup_all_defendants:
    for p in punctuation:
        if p in d:
            used_punctuations[p] += 1
used_punctuations

defaultdict(int,
            {'-': 470,
             "'": 101,
             '.': 26,
             '&': 4,
             '(': 10,
             ')': 10,
             ',': 16,
             '/': 7,
             '`': 3,
             ';': 1,
             '=': 1})

In [259]:
for d in dedup_all_defendants:
    if '.' in d:  # used_punctuations.keys()
        print(d)

r.tudor investments llc
m. whitehead
w. billy king
dr. robbins bibbs
j.r. benton
ms. jordan arbo
wanderjaunt inc.
dr. ludmila pyter
joh.shay tinsley
dennard higgins sr.
v. lachaal
v. audreana bloodsaw
venture well drilling, pump co. llc
lene st. john
bader a. alkhaldi
r. trep thompson
samantha h.
katie st. louis
joshua st. clair
.99 super center ymas
estate of patricia a. fautenberry
the estate of joyce m. doran
skilah arnol.d
wanderjaunt, inc.
christopher st. armour
charles simpson jr.


In [261]:
# names longer than 5 tokens
# companies: llc, co.
for d in dedup_all_defendants:
    if len(d.split(' ')) > 5:
        print(d)

the estate of george speck, the estate of ethel speck
antonio de jesus corona et al
the estate of leslie r weinberger
venture well drilling, pump co. llc
the estate of jim bohanon, sonia m carano
the estate of joyce m. doran
bp saratoga ridge ut 4 llc
all tenents and occupants and homeless
the estate of john michael beausoleil
med shape weight loss clinic, llc
miram hutchinson irrevocable living trust march
jason coleman / sweetwater behavioral care


In [466]:
# name length
defendant_name_len = [len(d.split(' ')) for d in dedup_all_defendants]
defendant_name_len_counter = Counter(defendant_name_len)
# length, total, percentage
sorted([(k, v, 100.0 * v / len(defendant_name_len)) for k, v in defendant_name_len_counter.items()], key=lambda x: x[1], reverse=True)

[(2, 37433, 97.0018139414356),
 (3, 943, 2.443638248250842),
 (4, 162, 0.4197978750971754),
 (5, 38, 0.09847110650427572),
 (6, 10, 0.025913449080072558),
 (1, 2, 0.005182689816014512),
 (10, 1, 0.002591344908007256),
 (8, 1, 0.002591344908007256)]

In [222]:
dedup_all_defendants_list = list(dedup_all_defendants)

# tokenization and blocking
def tokenize(name):
    return list(set(name.split(' ')) - set(punctuation))

defendant_ii = defaultdict(set)
for idx, name in enumerate(dedup_all_defendants_list):
    tokens = tokenize(name)
    for t in tokens:
        defendant_ii[t].add(idx)

In [223]:
len(defendant_ii.keys())

22761

In [224]:
# # of full comparison, 3 of blocked comps
full_comps = len(dedup_all_defendants) * (len(dedup_all_defendants)-1)
blocked_comps = sum([len(v)*(len(v)-1) for k, v in defendant_ii.items()])
full_comps, blocked_comps

(1489149510, 3721514)

In [225]:
# need a more effective blocking method? RR is already:
f'RR: {round(100 * (1 - blocked_comps / full_comps), 2)}%'

'RR: 99.75%'

In [239]:
import rltk
for idx, (k, v) in enumerate(defendant_ii.items()):
    v = list(v)
    for i in range(len(v)):
        for j in range(i+1, len(v)):
            r1 = dedup_all_defendants_list[v[i]]
            r2 = dedup_all_defendants_list[v[j]]
            sim = rltk.monge_elkan_similarity(tokenize(r1), tokenize(r2))
            if sim > 0.9:
                print(r1, '|', r2, '|', sim)
    if idx > 5:
        break

xavier piedra | xavier pierce | 0.9222222222222222
xavier pierce | xavier perez | 0.92
kristin bell | tristen bell | 0.9047619047619049
kristin bell | christina bell | 0.9206349206349207
branden bell | breanna bell | 0.9396825396825397
lorene bell | bell leon | 0.9125
lorene bell | lawrence bell | 0.9187500000000001
christina bell | christopher bell | 0.9212121212121211
ajia bell | ariya bell | 0.9025
vanessa bell | jhanessa bell | 0.9345238095238095
steve bell | steven bell | 0.9833333333333334
jhanessa bell | jessica bell | 0.9008928571428572
aaron johnson | aaron jones | 0.9161904761904762
aaron marin | aaron martin | 0.9805555555555556
aaron burton | aaron brown | 0.92
aaron stone | aaron sutton | 0.92
aaron stone | aaron stowers | 0.9266666666666666
aaron morand | aaron jordan | 0.9111111111111112
aaron williamson | aaron yates williams | 0.98
aaron anderson | aaron zanders | 0.9345238095238095
aaron mcmillan | aaron mcmillian | 0.9888888888888889
aaron browning | aaron brown | 0.

## Address

In [244]:
df_2020[df_2020['Case Number']=='CC2020000495000']['Address'], df_2020[df_2020['Case Number']=='CC2020000495000']['Defendant1 Address']

(1    5102 E PIEDMONT ROAD 2244, PHOENIX
 Name: Address, dtype: object,
 1    5102 E PIEDMONT ROAD, PHOENIX  AZ
 Name: Defendant1 Address, dtype: object)

In [10]:
df_2020[df_2020['Address'] == df_2020['Defendant1 Address']]

,File Date,Case Number,SubCategory,Immediate,Court,Case Status,Case Status Date,Monetary Judgment,Judgment For,Address,...,Def2 Ph,Plaintiff Name,Plaintiff1 Address,Plaint1 Zip,Plaintiff1 Email,Plaintiff1 Ph,Plt FTA?,Plaintiff2 Name,Plaintiff2 Email,Plaintiff2 Ph


In [11]:
df_address = df_2020.apply(lambda x: x['Address'][:10] if not pd.isna(x['Address']) else None, axis=1)
df_defendant1_address = df_2020.apply(lambda x: x['Defendant1 Address'][:10] if not pd.isna(x['Defendant1 Address']) else None, axis=1)

In [12]:
# how many cases whose first ten letters of df_address 
# are the same as the first ten letters of df_defendant1_address.
df_2020[df_address == df_defendant1_address]['Case Number'].shape[0]

34218

In [13]:
# how many of them are different
df_2020['Case Number'].shape[0] - df_2020[df_address == df_defendant1_address]['Case Number'].shape[0]

3542

In [14]:
# df_compare = pd.concat(
#     {'Address': df_address, 'Defendant1 Address': df_defendant1_address}, axis=1)
# df_compare = df_compare.reset_index(drop=True)
# df_compare

In [17]:
all_addresses = set(filter(lambda x: not pd.isna(x), df_2020['Address'].to_list()))
all_addresses

{'829 E TURNEY AVE UNIT 1, PHX',
 '4141 N 55TH AVE  1250, PHOENIX',
 '420 N Scottsdale Rd 6024, Tempe',
 '1450 S COOPER RD 2059, CHANDLER',
 '13421 N 43rd Ave #3067, Phoenix',
 '1460 E BELL ROAD 2086-1, PHOENIX',
 '4221 W DUNLAP AVE 235, PHX',
 '4630 S Lakeshore Drive 125, Tempe',
 '13300 N 88TH AVE UNIT 3014, PEORIA',
 '5150 W EUGIE AVENUE 2011, GLENDALE',
 '17600 N. 79TH AVE. (GATE CODE #1606) UNIT #508, GLENDALE',
 '1119 N 9TH STREET, PHOENIX',
 '538 N 55TH AVE 1031, GLENDALE',
 '6755 N 83RD AVE 15205, GLENDALE',
 '11459 NORTH 28TH DRIVE 3002, PHOENIX',
 '6505 E OSBORN RD UNIT 215-1, SCOTTSDALE',
 '34000 N 27th Dr #3116, Phoenix',
 '2145 W BROADWAY ROAD 251, MESA',
 '6901 W MCDOWELL RD 8204, PHOENIX',
 '7340 E LEGACY BLVD UNIT J1004, SCOTTSDALE',
 '24840 W DOVE TRAIL, BUCKEYE',
 '1033 S. LONGMORE ST #1094, MESA',
 '8520 N 54TH DRIVE, GLENDALE',
 '4002 E SOUTHERN AVE UNIT 249, PHOENIX',
 '1829 N 45th Ave Unit# 2128-2, Phoenix',
 '8600 E BROADWAY RD  #0109, MESA',
 '7007 W INDIAN SCHO

In [173]:
usaddress.tag('7375 N ZANJERO BLVD 5-1009, GLENDALE')[0]

OrderedDict([('AddressNumber', '7375'),
             ('StreetNamePreDirectional', 'N'),
             ('StreetName', 'ZANJERO'),
             ('StreetNamePostType', 'BLVD'),
             ('OccupancyIdentifier', '5-1009'),
             ('PlaceName', 'GLENDALE')])

In [216]:
usaddress.parse('1401 NORTH 43RD AVENUE  UNIT # 2012, PHOENIX')

[('1401', 'AddressNumber'),
 ('NORTH', 'StreetNamePreDirectional'),
 ('43RD', 'StreetName'),
 ('AVENUE', 'StreetNamePostType'),
 ('UNIT', 'OccupancyType'),
 ('#', 'OccupancyIdentifier'),
 ('2012,', 'OccupancyIdentifier'),
 ('PHOENIX', 'PlaceName')]

In [262]:
address_tag_content = defaultdict(set)
address_parsed = []
# for addr in all_addresses:
#     try:
#         parsed_addr = usaddress.tag(addr)[0]
#     except usaddress.RepeatedLabelError as e:
#         print('-' * 10)
#         print('RepeatedLabelError:', e)
#         print('Original:', e.original_string)
#         print('Parsed:', e.parsed_string)
#     for content, tag in parsed_addr.items():
#         address_tag_content[tag].add(content)

for addr in all_addresses:
    parsed_addr = usaddress.parse(addr)
    for content, tag in parsed_addr:
        address_tag_content[tag].add(content)
    address_parsed.append({'original': addr, **{pa[1]: pa[0] for pa in parsed_addr}})

In [263]:
address_tag_content.keys()

dict_keys(['AddressNumber', 'StreetNamePreDirectional', 'StreetName', 'StreetNamePostType', 'OccupancyType', 'OccupancyIdentifier', 'PlaceName', 'SubaddressType', 'SubaddressIdentifier', 'ZipCode', 'CountryName', 'StreetNamePreType', 'StreetNamePostDirectional', 'StateName', 'AddressNumberPrefix', 'USPSBoxType', 'USPSBoxID', 'AddressNumberSuffix', 'USPSBoxGroupType', 'BuildingName', 'Recipient', 'IntersectionSeparator'])

In [264]:
# address_tag_content['StreetNamePreDirectional']

In [265]:
# address_tag_content['StreetNamePostType']

In [81]:
df_address = pd.DataFrame(address_parsed)
df_address

,original,AddressNumber,StreetNamePreDirectional,StreetName,StreetNamePostType,OccupancyType,OccupancyIdentifier,PlaceName,SubaddressType,SubaddressIdentifier,...,StreetNamePostDirectional,StateName,AddressNumberPrefix,USPSBoxType,USPSBoxID,AddressNumberSuffix,USPSBoxGroupType,BuildingName,Recipient,IntersectionSeparator
0,"829 E TURNEY AVE UNIT 1, PHX",829,E,TURNEY,AVE,UNIT,PHX,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"4141 N 55TH AVE 1250, PHOENIX",4141,N,55TH,AVE,NaN,"1250,",PHOENIX,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"420 N Scottsdale Rd 6024, Tempe",420,N,Scottsdale,Rd,NaN,"6024,",Tempe,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"1450 S COOPER RD 2059, CHANDLER",1450,S,COOPER,RD,NaN,"2059,",CHANDLER,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"13421 N 43rd Ave #3067, Phoenix",13421,N,43rd,Ave,NaN,"3067,",Phoenix,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32342,"12930 W PARADISE DR, EL MIRAGE",12930,W,PARADISE,"DR,",NaN,NaN,MIRAGE,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32343,"1330 W BROADWAY RD A-104, TEMPE",1330,W,BROADWAY,RD,TEMPE,"A-104,",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32344,"5125 N. 40TH ST. F216, PHOENIX",5125,N.,40TH,ST.,NaN,"F216,",PHOENIX,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32345,"1950 N 43rd Ave Unit# 1047-1, Phoenix",1950,N,43rd,Ave,Unit,"1047-1,",Phoenix,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [114]:
random.seed(2021)
df_address_samples = pd.DataFrame.from_dict(
    {tag: [random.sample(content, k=min(5, len(content)))] for tag, content in address_tag_content.items()},
    orient='index',
    columns=['Samples']
)

In [115]:
df_address_stat = pd.DataFrame(df_address.count(), columns=['Count'])
df_address_stat['%'] = df_address_stat.apply(lambda x: round(100.0 * x / df_address.shape[0], 2))
df_address_stat = df_address_stat.join(df_address_samples)
df_address_stat.sort_values(by=['Count'], ascending=False)

,Count,%,Samples
original,32347,100.00,NaN
StreetName,32334,99.96,"[SOL,, LONGVIEW, 209th, SPANISH, 92ST]"
AddressNumber,32332,99.95,"[10769, 1325, 22629, 15431W, 31155]"
StreetNamePreDirectional,32099,99.23,"[NE, W,, w., SOUTH, M.]"
PlaceName,30608,94.62,"[wickenberg, PHOOENIX, PHOENNIX, MIRAGE, mesa]"
StreetNamePostType,30549,94.44,"[road,, AVENUE,, RE, BLVD.,, pl,]"
OccupancyIdentifier,26200,81.00,"[189A,, 1011,, 2144,, E105,, 7724-07,]"
OccupancyType,9036,27.93,"[tempe, OHX, UNITN, Apt., Drve]"
StreetNamePreType,821,2.54,"[BRILL, McDowell, ROAD, CAMINO, KATHLEEN]"
StreetNamePostDirectional,153,0.47,"[E,, WEST, SE,, ve, EAST]"


In [120]:
# need data cleaning
for addr in address_parsed:
    if 'AddressNumber' in addr and addr['AddressNumber'] == '15431W':
        print(addr)
        break

{'original': '15431W DESERT MIRAGE DR, SURPRISE', 'AddressNumber': '15431W', 'StreetName': 'MIRAGE', 'StreetNamePostType': 'DR,', 'PlaceName': 'SURPRISE'}


In [269]:
for addr in address_parsed:
    if 'AddressNumber' in addr and not addr['AddressNumber'].isnumeric():
        print(addr['AddressNumber'], '|', addr['original'])

B-13413 | B-13413 N 35TH AVE 2025B, PHOENIX
B-13413 | B-13413 NORTH 35TH AVENUE 1027B, PHOENIX
2634N | 2634N 51ST AVE 144, PHOENIX
9920W. | 9920W. CAMELBACK RD., UNIT#1166, PHOENIX
B-13413 | B-13413 NORTH 35TH AVE 2075B, PHOENIX
7102N | 7102N 43RD AVE 252, GLENDALE
7102W | 7102W N 43RD AVE 310, GLENDALE
B-13413 | B-13413 N 35TH AVE 2098B, PHOENIX
B-13413 | B-13413 N 35TH AVE 1049B, PHOENIX
19+800 | 19+800 N 7TH ST #1016, PHOENIX
B-13413 | B-13413 N 35 AVE 2039B, PHOENIX
5632W | 5632W MYRTLE AVE 105, GLENDALE
B13413 | B13413 N 35TH AVE 2025B, PHOENIX
B-13413 | B-13413 N 35TH AVE 1090B, PHOENIX
B-13413 | B-13413 NORTH 35TH AVENUE 2064B, PHOENIX
B-13413 | B-13413 N 35 AVE 2026B, PHOENIX
B-13413 | B-13413 NORTH 35TH AVENUE 1090B, PHOENIX
13229+ | 13229+ SOUTH 48TH STREET UNIT A1035, PHOENIX
B-13413 | B-13413 N 35TH AVE 1017B, PHOENIX
B-13413 | B-13413 N 35TH AVE APT 1090B, PHOENIX
B-13413 | B-13413 NORTH 35TH AVENUE 1001B, PHOENIX
52F | 52F SOUTH BEVERLY STREET #B04, MESA
4114W | 4114W SAI

In [274]:
Counter(df_address['StreetNamePreDirectional'].to_list())

Counter({'E': 4968,
         'N': 7956,
         'S': 2661,
         'W': 8424,
         'N.': 1138,
         'NORTH': 1292,
         'S.': 587,
         'EAST': 499,
         'W.': 1533,
         'E.': 607,
         'WEST': 1272,
         'SOUTH': 624,
         'E,': 7,
         'North': 209,
         nan: 248,
         'West': 62,
         'n': 51,
         'e': 12,
         'East': 61,
         's': 7,
         'N,': 8,
         'w': 52,
         '2': 2,
         'north': 6,
         'M': 2,
         'n.': 2,
         'NW': 19,
         'WEST.': 1,
         'South': 7,
         'W,': 9,
         'E..': 1,
         'SW': 1,
         'W/': 3,
         'west': 5,
         'w.': 5,
         'NN': 1,
         'NOR': 1,
         'NE': 1,
         'N..': 1,
         'M.': 1,
         'B': 1})

In [304]:
place_names = list(set(df_address['PlaceName'].to_list()))
place_names.remove(np.nan)
place_name = list(set(map(lambda x: x.lower(), place_names)))
sorted(place_name)

['(front),',
 '(garage),',
 '(rear),',
 '(studio),',
 'a),',
 'anthem',
 'arlington',
 'avodale',
 'avondal',
 'avondale',
 'avondale,',
 'avondle',
 'bend',
 'buckeye',
 'c,',
 'camdenton',
 'camille,',
 'cashion',
 'chaandler',
 'chandelr',
 'chandfler',
 'chandler',
 'chandlerl',
 'circle,',
 'city',
 'creek',
 'creel',
 'd,',
 'erton',
 'esa',
 'front,',
 'garfield,',
 'gilbert',
 'gilbertq',
 'glenale',
 'glendal',
 'glendale',
 'glendale,',
 'glendlae',
 'glendle',
 'glenview',
 'godyear',
 'gooddyear',
 'goodyear',
 'guadalupe',
 'hills',
 'hoenix',
 'house),',
 'house,',
 'howell',
 'jpemox',
 'junction',
 'lakes',
 'laveen',
 'litchfield',
 'loft,',
 'lorenzo,',
 'mea',
 'mesa',
 'mesaa',
 'messa',
 'mirage',
 'mirgage',
 'morristown',
 'mriage',
 'na,',
 'ndale',
 'nix',
 'north,',
 'oungtown',
 'park',
 'payson',
 'peooria',
 'peoria',
 'phenix',
 'pheonix',
 'phhoenix',
 'phioenix',
 'phoeinx',
 'phoeix',
 'phoeni',
 'phoenic',
 'phoeniex',
 'phoenix',
 'phoenix,',
 'phoeni

## ER for address

In [396]:
# normalization
# https://www.maricopa.gov/5312/Cities-Towns
# std_city_names = {'Apache Junction', 'Avondale', 'Buckeye', 'Carefree', 'Cave Creek', 'Chandler', 'El Mirage', 'Fountain Hills', 'Gila Bend', 'Gilbert', 'Glendale', 'Goodyear', 'Guadalupe', 'Litchfield Park', 'Mesa', 'Paradise Valley', 'Peoria', 'Phoenix', 'Queen Creek', 'Scottsdale', 'Sun City', 'Sun City West', 'Surprise', 'Tempe', 'Tolleson', 'Wickenburg', 'Youngtown'}
# std_city_names = {n.lower() for n in std_city_names}
std_city_names = {'avodale': 'avondale', 'avondal': 'avondale', 'avondale': 'avondale', 'avondale,': 'avondale', 'avondle': 'avondale', 'buckeye': 'buckeye', 'chaandler': 'chandler', 'chandelr': 'chandler', 'chandfler': 'chandler', 'chandler': 'chandler', 'chandlerl': 'chandler', 'gilbert': 'gilbert', 'gilbertq': 'gilbert', 'glenale': 'glendale', 'glendal': 'glendale', 'glendale': 'glendale', 'glendale': 'glendale', 'glendlae': 'glendale', 'glendle': 'glendale', 'godyear': 'goodyear', 'gooddyear': 'goodyear', 'goodyear': 'goodyear', 'mea': 'mesa', 'mesa': 'goodyear', 'mesaa': 'goodyear', 'messa': 'goodyear', 'peooria': 'peoria', 'peoria': 'peoria', 'phenix': 'phoenix', 'pheonix': 'phoenix', 'phhoenix': 'phoenix', 'phioenix': 'phoenix', 'phoeinx': 'phoenix', 'phoeix': 'phoenix', 'phoeni': 'phoenix', 'phoenic': 'phoenix', 'phoeniex': 'phoenix', 'phoenix': 'phoenix', 'phoenix': 'phoenix', 'phoenixm': 'phoenix', 'phoenixq': 'phoenix', 'phoenixx': 'phoenix', 'phoeniz': 'phoenix', 'phoennix': 'phoenix', 'phoenuix': 'phoenix', 'phoenx': 'phoenix', 'phoenxi': 'phoenix', 'phoeonix': 'phoenix', 'phoienix': 'phoenix', 'phoneix': 'phoenix', 'phoniex': 'phoenix', 'phooenix': 'phoenix', 'phornix': 'phoenix', 'phpoenix': 'phoenix', 'phx': 'phoenix', 'phx': 'phoenix', 'poenix': 'phoenix', 'pohenix': 'phoenix', 'pyhoenix': 'phoenix', 'scottdale': 'scottsdale', 'scottsadale': 'scottsdale', 'scottsdake': 'scottsdale', 'scottsdal': 'scottsdale', 'scottsdale': 'scottsdale', 'scottsdale': 'scottsdale', 'scttsdale': 'scottsdale', 'slurprise': 'scottsdale', 'surprise': 'surprise', 'surprsie': 'surprise', 'temep': 'tempe', 'temoe': 'tempe', 'tempe': 'tempe', 'tenoe': 'tempe', 'tollesin': 'tolleson', 'tolleson': 'tolleson', 'tollseon': 'tolleson', 'wickenberg': 'wickenburg', 'wickenburg': 'wickenburg'}
# https://pe.usps.com/text/pub28/28c2_toc.htm
std_street_name_directional = {'EAST': 'E', 'NORTH': 'N', 'NORTHEAST': 'NE', 'NORTHWEST': 'NW', 'SOUTH': 'S', 'SOUTHEAST': 'SE', 'SOUTHWEST': 'SW', 'WEST': 'W'}
std_street_name_directional = {k.lower(): v.lower() for k, v in std_street_name_directional.items()}

def remove_punctuations(n):
    return n.translate(str.maketrans('', '', punctuation))

def normalize_street_name_directional(n):
    n = n.lower()
    n = remove_punctuations(n)
    return std_street_name_directional.get(n, n)

def normalize_city_name(n):
    n = n.lower()
    n = remove_punctuations(n)
    return std_city_names.get(n, n)

In [397]:
cleaned_addresses = []
for idx, addr in enumerate(address_parsed):
    cleaned_addr = addr.get('AddressNumber', '').lower() + ' '\
        + normalize_street_name_directional(addr.get('StreetNamePreDirectional', '').lower()) + ' '\
        + addr.get('StreetName', '').lower() + ' '\
        + addr.get('StreetNamePostType', '').lower() + ' '\
        + normalize_street_name_directional(addr.get('StreetNamePostDirectional', '').lower()) + ' '\
        + normalize_city_name(addr.get('PlaceName', '').lower())
    
    cleaned_addr = cleaned_addr.replace(',', ' ').replace('.', '')
    cleaned_addresses.append(cleaned_addr)
    if idx < 20:
        print(cleaned_addr)

829 e turney ave  
4141 n 55th ave  phoenix
420 n scottsdale rd  tempe
1450 s cooper rd  chandler
13421 n 43rd ave  phoenix
1460 e bell road  phoenix
4221 w dunlap ave  phoenix
4630 s lakeshore drive  tempe
13300 n 88th ave  peoria
5150 w eugie avenue  glendale
17600 n 79th ave  glendale
1119 n 9th street   phoenix
538 n 55th ave  glendale
6755 n 83rd ave  
11459 n 28th drive  phoenix
6505 e osborn rd  scottsdale
34000 n 27th dr  phoenix
2145 w broadway road  goodyear
6901 w mcdowell rd  phoenix
7340 e legacy blvd  scottsdale


In [398]:
street_names = []
for addr in address_parsed:
    street_name = addr.get('StreetName')
    if not street_name:
        print(addr)
        continue
    street_names.append(street_name.lower())

{'original': 'PO BOX 263, AVONDALE', 'USPSBoxType': 'BOX', 'USPSBoxID': '263,', 'PlaceName': 'AVONDALE'}
{'original': 'PO BOX 41941, PHOENIX', 'USPSBoxType': 'BOX', 'USPSBoxID': '41941,', 'PlaceName': 'PHOENIX'}
{'original': 'P O BOX 34772, PHOENIX', 'USPSBoxType': 'BOX', 'USPSBoxID': '34772,', 'PlaceName': 'PHOENIX'}
{'original': 'NA, NA', 'PlaceName': 'NA,', 'StateName': 'NA'}
{'original': 'po box 12197, Glendale', 'USPSBoxType': 'box', 'USPSBoxID': '12197,', 'PlaceName': 'Glendale'}
{'original': 'PO Box 28171, Scottsdale', 'USPSBoxType': 'Box', 'USPSBoxID': '28171,', 'PlaceName': 'Scottsdale'}
{'original': 'PO Box 5791, Peoria', 'USPSBoxType': 'Box', 'USPSBoxID': '5791,', 'PlaceName': 'Peoria'}
{'original': 'P O BOX 25414, SCOTTSDALE', 'USPSBoxType': 'BOX', 'USPSBoxID': '25414,', 'PlaceName': 'SCOTTSDALE'}
{'original': '16220 NORTH 7TH 3436, PHOENIX', 'AddressNumber': '16220', 'StreetNamePreDirectional': 'NORTH', 'SubaddressType': '7TH', 'SubaddressIdentifier': '3436,', 'PlaceName':

In [399]:
# duplicated street names
len(street_names), len(set(street_names))

(32334, 2653)

In [400]:
# length of street name (# of words)
set([len(name.split(' ')) for name in street_names])

{1}

In [401]:
# def qgram_tokenizer(n, s, place_holder=' ', padded=False):
#     if len(s) == 0:
#         return []
#     if padded:
#         pad = place_holder * (n - 1)
#         s = pad + s + pad
#     s = s.split(' ')
#     s = place_holder.join(s)
#     if len(s) < n:
#         return [s]
#     return [s[i:i + n] for i in range(len(s) - n + 1)]

In [402]:
# inverted index
street_name_ii = defaultdict(set)
for idx, addr in enumerate(address_parsed):
    street_name = addr.get('StreetName')
    if not street_name:
        continue
    street_name_ii[street_name.lower()].add(idx)

In [403]:
# candidate pairs
cand_pairs = set()
for _, ids in street_name_ii.items():
    for p in list(combinations(ids, 2)):
        cand_pairs.add(p)
len(cand_pairs)

3255242

In [425]:
# tfidf score
from rltk.similarity import TF_IDF

tfidf = TF_IDF()

for idx, addr in enumerate(cleaned_addresses):
    addr_tokens = list(filter(lambda x: x != '', addr.split(' ')))
#     print(addr_tokens)
    tfidf.add_document(idx, addr_tokens)
    if idx < 20:
        print(addr_tokens)
tfidf.pre_compute()

['829', 'e', 'turney', 'ave']
['4141', 'n', '55th', 'ave', 'phoenix']
['420', 'n', 'scottsdale', 'rd', 'tempe']
['1450', 's', 'cooper', 'rd', 'chandler']
['13421', 'n', '43rd', 'ave', 'phoenix']
['1460', 'e', 'bell', 'road', 'phoenix']
['4221', 'w', 'dunlap', 'ave', 'phoenix']
['4630', 's', 'lakeshore', 'drive', 'tempe']
['13300', 'n', '88th', 'ave', 'peoria']
['5150', 'w', 'eugie', 'avenue', 'glendale']
['17600', 'n', '79th', 'ave', 'glendale']
['1119', 'n', '9th', 'street', 'phoenix']
['538', 'n', '55th', 'ave', 'glendale']
['6755', 'n', '83rd', 'ave']
['11459', 'n', '28th', 'drive', 'phoenix']
['6505', 'e', 'osborn', 'rd', 'scottsdale']
['34000', 'n', '27th', 'dr', 'phoenix']
['2145', 'w', 'broadway', 'road', 'goodyear']
['6901', 'w', 'mcdowell', 'rd', 'phoenix']
['7340', 'e', 'legacy', 'blvd', 'scottsdale']


In [426]:
# adjust idf score
avg_idf = 1.0 * sum(tfidf.idf.values()) / len(tfidf.idf.values())
print(avg_idf)
for n in std_street_name_directional.values():
    tfidf.idf[n.lower()] = avg_idf
for n in std_city_names.values():
    tfidf.idf[n.lower()] = avg_idf

19382.35582212394


In [433]:
# er
pairs = []
for idx, cp in enumerate(tqdm(cand_pairs)):
#     if idx > 1000:
#         break
    # force the addr num to be the same
    addr_num1 = address_parsed[cp[0]].get('AddressNumber')
    addr_num2 = address_parsed[cp[1]].get('AddressNumber')
    if addr_num1 and addr_num2 and addr_num1 != addr_num2:
        continue
        
    sim = tfidf.similarity(*cp)
    if sim > 0.8:
        pairs.append((*cp, sim))
#         print(sim, '|', address_parsed[cp[0]]['original'], '|', address_parsed[cp[1]]['original'])

100%|██████████| 3255242/3255242 [00:07<00:00, 440238.56it/s]


In [434]:
len(pairs), len(pairs) - len(cand_pairs)

(366163, -2889079)

In [435]:
# clustering
nx_graph = nx.Graph()
nx_graph.add_edges_from([(p[0], p[1]) for p in pairs])
address_clusters = list(nx.connected_components(nx_graph))
len(address_clusters)

2808

In [438]:
for cid in random.sample(range(len(address_clusters)), 10):
    for idx, id_ in enumerate(address_clusters[cid]):
        if idx > 20:
            break
        print(address_parsed[id_]['original'])
    print('-' * 10)

6802 W. GLENDALE AVENUE 6, GLENDALE
6802 W. GLENDALE AVE 21, GLENDALE
----------
2661 E AUGUSTA AVE, GILBERT
2661 E AUGUSTA AVENUE, GILBERT
2661 E Augusta Ave, Gilbert
----------
7001 E PALM LANE #237, SCOTTSDALE
7001 E PALM LN UNIT #233, SCOTTSDALE
7001 E PALM LN UNIT #116, SCOTTSDALE
7001 E PALM LN UNIT #146, SCOTTSDALE
7001 E Palm Ln #230, Scottsdale
----------
1901 E Apache Blvd #377, Tempe
1901 E Apache Blvd #343, Tempe
1901 E APACHE BLVD, UNIT# 364, TEMPE
1901 E APACHE BLVD, UNIT# 325, TEMPE
1901 E Apache Blvd #128, Tempe
1901 E APACHE BLVD, UNIT # 107, TEMPE
1901 E Apache Blvd #308, Tempe
1901 E Apache Blvd #292, Tempe
1901 E Apache Blvd #204, Tempe
1901 E APACHE BLVD UNIT 192, TEMPE
1901 E Apache Blvd #334, Tempe
1901 E Apache Blvd  Unit 178, Tempe
1901 E Apache blvd #209, Tempe
1901 E Apache Blvd #392, Tempe
1901 E Apache Blvd #389, Tempe
1901 E APACHE BLVD UNIT 282, TEMPE
1901 E. APACHE BLVD UNIT 390, TEMPE
1901 E Apache Blvd #387, Tempe
1901 E Apache Blvd Unit 270, Tempe
190

In [431]:
# tfidf.idf['w'], tfidf.idf['e'], tfidf.idf['west'], tfidf.idf['nw']

In [432]:
# tfidf.idf['2340'], tfidf.idf['6230']

### complete output to file

In [721]:
# 1st-level id to address mapping
address_list = []
for idx, row in df_2020[['Address', 'Case Number']].dropna().iterrows():
    id_ = row['Case Number']
    addr = row['Address']
    if not addr:
        continue
    address_list.append((id_, addr.strip()))
len(address_list)

34355

In [722]:
# 2nd-level id to address mapping
dedup_address = defaultdict(set)
for idx, addr in enumerate(address_list):
    dedup_address[addr[1].lower()].add(idx)

sec_id_mapping = []
dedup_address_list = []
for n, ids in dedup_address.items():
    sec_id_mapping.append(ids)
    dedup_address_list.append(n)
len(dedup_address_list)

32106

In [723]:
address_tag_content = defaultdict(set)
address_parsed = []

for addr in dedup_address_list:
    parsed_addr = usaddress.parse(addr)
    for content, tag in parsed_addr:
        address_tag_content[tag].add(content)
    address_parsed.append({'original': addr, **{pa[1]: pa[0] for pa in parsed_addr}})

In [724]:
cleaned_addresses = []
for idx, addr in enumerate(address_parsed):
    cleaned_addr = addr.get('AddressNumber', '').lower() + ' '\
        + normalize_street_name_directional(addr.get('StreetNamePreDirectional', '').lower()) + ' '\
        + addr.get('StreetName', '').lower() + ' '\
        + addr.get('StreetNamePostType', '').lower() + ' '\
        + normalize_street_name_directional(addr.get('StreetNamePostDirectional', '').lower()) + ' '\
        + normalize_city_name(addr.get('PlaceName', '').lower())
    
    cleaned_addr = cleaned_addr.replace(',', ' ').replace('.', '')
    cleaned_addresses.append(cleaned_addr)
    if idx < 20:
        print(cleaned_addr)

10751 w davis lane   avondale
5102 e 2244    phoenix
3800 w chandler blvd  chandler
2800 n arizona ave  chandler
1720 e broadway rd  tempe
3950 w chandler blvd  chandler
6301 n 64th dr  phoenix
12175 w mcdowell rd  avondale
4030 n 44th ave  phoenix
8092 w paradise ln  peoria
10030 n 43rd avenue  glendale
5215 w peoria ave  glendale
42405 n town court   anthem
8650 w peoria ave  peoria
18330 n 79th ave  glendale
2140 e broadway road  phoenix
3134 s market st  gilbert
22650 w cocopah st   buckeye
1725 s coronado road  gilbert
1960 w keating ave  goodyear


In [725]:
street_names = []
for addr in address_parsed:
    street_name = addr.get('StreetName')
    if not street_name:
        print(addr)
        continue
    street_names.append(street_name.lower())

{'original': 'po box 263, avondale', 'USPSBoxType': 'box', 'USPSBoxID': '263,', 'PlaceName': 'avondale'}
{'original': '1141 e, garfield, phoenix', 'AddressNumber': '1141', 'StreetNamePreDirectional': 'e,', 'PlaceName': 'garfield,', 'StateName': 'phoenix'}
{'original': 'empe, tempe', 'Recipient': 'tempe'}
{'original': '16220 north 7th 3436, phoenix', 'AddressNumber': '16220', 'StreetNamePreDirectional': 'north', 'SubaddressType': '7th', 'SubaddressIdentifier': '3436,', 'PlaceName': 'phoenix'}
{'original': '16220 north 7th 2172, phoenix', 'AddressNumber': '16220', 'StreetNamePreDirectional': 'north', 'SubaddressType': '7th', 'SubaddressIdentifier': '2172,', 'PlaceName': 'phoenix'}
{'original': 'po box 41941, phoenix', 'USPSBoxType': 'box', 'USPSBoxID': '41941,', 'PlaceName': 'phoenix'}
{'original': 'po box 12197, glendale', 'USPSBoxType': 'box', 'USPSBoxID': '12197,', 'PlaceName': 'glendale'}
{'original': 'p o box 25414, scottsdale', 'USPSBoxType': 'box', 'USPSBoxID': '25414,', 'PlaceNam

In [726]:
# inverted index
street_name_ii = defaultdict(set)
for idx, addr in enumerate(address_parsed):
    street_name = addr.get('StreetName')
    if not street_name:
        continue
    street_name_ii[street_name.lower()].add(idx)

In [727]:
# candidate pairs
cand_pairs = set()
for _, ids in street_name_ii.items():
    for p in list(combinations(ids, 2)):
        cand_pairs.add(p)
len(cand_pairs)

3205757

In [728]:
# tfidf score
from rltk.similarity import TF_IDF

tfidf = TF_IDF()

for idx, addr in enumerate(cleaned_addresses):
    addr_tokens = list(filter(lambda x: x != '', addr.split(' ')))
#     print(addr_tokens)
    tfidf.add_document(idx, addr_tokens)
    if idx < 20:
        print(addr_tokens)
tfidf.pre_compute()

['10751', 'w', 'davis', 'lane', 'avondale']
['5102', 'e', '2244', 'phoenix']
['3800', 'w', 'chandler', 'blvd', 'chandler']
['2800', 'n', 'arizona', 'ave', 'chandler']
['1720', 'e', 'broadway', 'rd', 'tempe']
['3950', 'w', 'chandler', 'blvd', 'chandler']
['6301', 'n', '64th', 'dr', 'phoenix']
['12175', 'w', 'mcdowell', 'rd', 'avondale']
['4030', 'n', '44th', 'ave', 'phoenix']
['8092', 'w', 'paradise', 'ln', 'peoria']
['10030', 'n', '43rd', 'avenue', 'glendale']
['5215', 'w', 'peoria', 'ave', 'glendale']
['42405', 'n', 'town', 'court', 'anthem']
['8650', 'w', 'peoria', 'ave', 'peoria']
['18330', 'n', '79th', 'ave', 'glendale']
['2140', 'e', 'broadway', 'road', 'phoenix']
['3134', 's', 'market', 'st', 'gilbert']
['22650', 'w', 'cocopah', 'st', 'buckeye']
['1725', 's', 'coronado', 'road', 'gilbert']
['1960', 'w', 'keating', 'ave', 'goodyear']


In [729]:
# adjust idf score
avg_idf = 1.0 * sum(tfidf.idf.values()) / len(tfidf.idf.values())
print(avg_idf)
for n in std_street_name_directional.values():
    tfidf.idf[n.lower()] = avg_idf
for n in std_city_names.values():
    tfidf.idf[n.lower()] = avg_idf

19325.18109966905


In [730]:
# er
pairs = []
for idx, cp in enumerate(tqdm(cand_pairs)):
#     if idx > 1000:
#         break
    # force the addr num to be the same
    addr_num1 = address_parsed[cp[0]].get('AddressNumber')
    addr_num2 = address_parsed[cp[1]].get('AddressNumber')
    if addr_num1 and addr_num2 and addr_num1 != addr_num2:
        continue
        
    sim = tfidf.similarity(*cp)
    if sim > 0.8:
        pairs.append((*cp, sim))
#         print(sim, '|', address_parsed[cp[0]]['original'], '|', address_parsed[cp[1]]['original'])


100%|██████████| 3205757/3205757 [00:08<00:00, 396917.60it/s]


In [731]:
# clustering
nx_graph = nx.Graph()
nx_graph.add_edges_from([(p[0], p[1]) for p in pairs])
address_clusters = list(nx.connected_components(nx_graph))
len(address_clusters)

2749

In [732]:
with open('address_clusters_20210813.json', 'w') as f:
    output = []
    for c in address_clusters:
        obj = {}
        for id2 in c: # 2nd-level id
            for id1 in sec_id_mapping[id2]: # 1st-level id
                obj[address_list[id1][0]] = address_list[id1][1]
        output.append(obj)
    json.dump(output, f, indent=4)

## Plaintiffs

Related attributes: 'Plaintiff Name', 'Plaintiff1 Address', 'Plaint1 Zip', 'Plaintiff1 Email', 'Plaintiff1 Ph', 'Plt FTA?', 'Plaintiff2 Name', 'Plaintiff2 Email', 'Plaintiff2 Ph'

In [674]:
plaintiff1 = df_2020[['Plaintiff Name']].dropna()['Plaintiff Name'].tolist()
plaintiff2 = df_2020[['Plaintiff2 Name']].dropna()['Plaintiff2 Name'].tolist()
len(plaintiff1), len(plaintiff2), len(plaintiff1 + plaintiff2)

(37760, 1700, 39460)

In [675]:
dedup_plaintiff1 = set(map(lambda x: x.lower().strip(), plaintiff1))
dedup_plaintiff2 = set(map(lambda x: x.lower().strip(), plaintiff2))
dedup_all_plaintiffs = dedup_plaintiff1 | dedup_plaintiff2
len(dedup_plaintiff1), len(dedup_plaintiff2), len(dedup_all_plaintiffs)

(11397, 1004, 12046)

In [455]:
used_punctuations = defaultdict(int)
for d in dedup_all_plaintiffs:
    for p in punctuation:
        if p in d:
            used_punctuations[p] += 1
used_punctuations
sorted(used_punctuations.items(), key=lambda x: x[1], reverse=True)

[(',', 1486),
 ('/', 838),
 ('-', 514),
 ('&', 384),
 ('.', 351),
 ('(', 132),
 (')', 131),
 (';', 66),
 ('%', 48),
 ("'", 32),
 ('@', 20),
 (':', 15),
 ('#', 9),
 ('_', 3),
 ('`', 2),
 ('[', 2),
 ('*', 2),
 ('$', 1),
 ('=', 1)]

In [456]:
for d in dedup_all_plaintiffs:
    if '@' in d:
        print(d)

senderos @ south mountain llc
colonial grand@ old town scottsdale south
bmf iv az array south mountain llc dba array @ south mountain apts
bmf iv az lore south mountain llc dba lore @ south mountain apts
trailside@hermosa pointe apartments
trailside @ hermosa pointe apartments
trailside@ hermosa pointe apt
hardy drive investors llc dba ovation @ tempe apts
ka mre park @ 33rd llc
trailside @ hermosa pointe apts
tides @ south mountain c/o the robinson group
tides @ downtown tempe c/o the robinson group
tides @ south tempe c/o the robinson group
mid america apts lp dba residences @ fountainhead
ka mre park @ 33rd, llc
bmf iz az array south mountain llc dba array @ south mountain apts
ka mre park @33rd llc
hee vee @ vee lllp in care of metropolitan
tides @ tempe llc c/o the robinson group
deervalleyleasing@robinsongroupre.com c/o atrak investments llc


In [467]:
# name length
plaintiff_name_len = [len(d.split(' ')) for d in dedup_all_plaintiffs]
plaintiff_name_len_counter = Counter(plaintiff_name_len)
# length, total, percentage
sorted([(k, v, 100.0 * v / len(plaintiff_name_len)) for k, v in plaintiff_name_len_counter.items()], key=lambda x: x[1], reverse=True)

[(2, 3817, 31.686867009795783),
 (3, 2640, 21.91598870994521),
 (4, 2383, 19.782500415075543),
 (5, 1308, 10.858376224472854),
 (6, 601, 4.989208035862527),
 (7, 369, 3.063257512867342),
 (8, 354, 2.9387348497426533),
 (9, 204, 1.6935082184957662),
 (10, 157, 1.3033372073717417),
 (11, 76, 0.6309148264984227),
 (12, 43, 0.35696496762410757),
 (1, 41, 0.3403619458741491),
 (13, 19, 0.15772870662460567),
 (14, 14, 0.11622115224970944),
 (15, 8, 0.06641208699983397),
 (16, 4, 0.03320604349991699),
 (17, 3, 0.024904532624937737),
 (19, 2, 0.016603021749958494),
 (20, 1, 0.008301510874979247),
 (22, 1, 0.008301510874979247),
 (18, 1, 0.008301510874979247)]

In [461]:
# for d in dedup_all_plaintiffs:
#     if len(d.split(' ')) == 2:
#         print(d)

In [511]:
# token frequencies
company_name_tokens = defaultdict(int)
for d in dedup_all_plaintiffs:
    for t in d.split(' '):
        company_name_tokens[t] += 1

# len(company_name_tokens)
sorted(company_name_tokens.items(), key=lambda x: x[1], reverse=True)

[('llc', 4960),
 ('apartments', 787),
 ('lp', 677),
 ('c/o', 488),
 ('property', 445),
 ('properties', 435),
 ('inc', 360),
 ('of', 353),
 ('at', 340),
 ('&', 334),
 ('management', 327),
 ('', 311),
 ('in', 282),
 ('real', 281),
 ('care', 269),
 ('trust', 257),
 ('az', 248),
 ('phoenix', 247),
 ('investments', 247),
 ('homes', 245),
 ('apts', 235),
 ('estate', 227),
 ('realty', 216),
 ('holdings', 209),
 ('park', 202),
 ('the', 191),
 ('borrower', 189),
 ('on', 183),
 ('west', 171),
 ('owner', 165),
 ('residential', 153),
 ('partners', 145),
 ('family', 144),
 ('dba', 144),
 ('arizona', 142),
 ('san', 138),
 ('and', 134),
 ('group', 134),
 ('village', 122),
 ('ii', 121),
 ('progress', 120),
 ('tides', 119),
 ('valley', 116),
 ('apartments,', 112),
 ('invitation', 107),
 ('apartment', 103),
 ('desert', 103),
 ('home', 102),
 ('capital', 102),
 ('vista', 94),
 ('services', 89),
 ('enterprises', 88),
 ('creek', 85),
 ('tempe', 84),
 ('ave', 83),
 ('investment', 82),
 ('inc.', 81),
 ('hous

In [502]:
# token frequency, how many such tokens
# (1, 5765) -> 5765 tokens only appear once
sorted(Counter(company_name_tokens.values()).items(), key=lambda x: x[1], reverse=True)[:30]

[(1, 5765),
 (2, 1458),
 (3, 667),
 (4, 376),
 (5, 250),
 (6, 149),
 (7, 119),
 (8, 100),
 (9, 78),
 (11, 63),
 (10, 61),
 (13, 47),
 (12, 36),
 (14, 34),
 (15, 31),
 (16, 23),
 (17, 23),
 (18, 14),
 (19, 13),
 (23, 13),
 (24, 12),
 (21, 12),
 (20, 11),
 (33, 10),
 (25, 9),
 (27, 9),
 (22, 9),
 (37, 7),
 (40, 6),
 (29, 6)]

In [507]:
# for t, c in company_name_tokens.items():
#     if c == 8:
#         print(t)

In [678]:
# company name
# https://github.com/psolin/cleanco
cleanco_terms = cleanco.prepare_terms()
cleanco_type_sources = typesources()
cleaned_plaintiffs = []
dedup_all_plaintiffs_list = list(dedup_all_plaintiffs)
for idx, d in enumerate(dedup_all_plaintiffs):
    company_type = cleanco.matches(d, cleanco_type_sources)
    cleaned_name = cleanco.basename(d, cleanco_terms, prefix=False, middle=False, suffix=True)
    cleaned_plaintiffs.append(cleaned_name)
    if idx < 50:
        print(d, '|', cleaned_name, '|', company_type)

brianna stern | brianna stern | []
aldea at estrella falls  sig llc | aldea at estrella falls sig | ['Limited Liability Company']
law offices of scott m clark pc | law offices of scott m clark | []
court 64, llc | court 64 | ['Limited Liability Company']
aria on mill   pathfinder phoenix holdings inc. | aria on mill pathfinder phoenix holdings | ['Corporation']
pob llc | pob | ['Limited Liability Company']
judith sutton | judith sutton | []
cobblestone partners phoenix lp | cobblestone partners phoenix | ['Limited Partnership']
calcap properties inc / arches apartment homes | calcap properties inc / arches apartment homes | ['Corporation']
mc connel family trust | mc connel family trust | []
tempe vista llc | tempe vista | ['Limited Liability Company']
carlos duarte | carlos duarte | []
christine miles | christine miles | []
golden mouse enterprises llc | golden mouse enterprises | ['Limited Liability Company']
phoenix 83rd ave apartmrnts lp | phoenix 83rd ave apartmrnts | ['Limited Pa

In [681]:
# inverted index
plaintiff_ii = defaultdict(set)
plaintiff_tokens = []
for idx, name in enumerate(cleaned_plaintiffs):
    tokens = list(filter(lambda x: x != '' and x not in punctuation, name.split(' ')))
    plaintiff_tokens.append(tokens)
    for t in tokens:
        plaintiff_ii[t].add(idx)
len(plaintiff_ii)

9268

In [682]:
# candidate pairs
cand_pairs = set()
for _, ids in plaintiff_ii.items():
    for p in list(combinations(ids, 2)):
        cand_pairs.add(p)
len(cand_pairs)

1685978

In [628]:
# tfidf score
from rltk.similarity import TF_IDF

tfidf = TF_IDF()

for idx, name in enumerate(cleaned_plaintiffs):
    tfidf.add_document(idx, plaintiff_tokens[idx])
    if idx < 20:
        print(plaintiff_tokens[idx])
tfidf.pre_compute(math_log=False)

['brianna', 'stern']
['aldea', 'at', 'estrella', 'falls', 'sig']
['law', 'offices', 'of', 'scott', 'm', 'clark']
['court', '64']
['aria', 'on', 'mill', 'pathfinder', 'phoenix', 'holdings']
['pob']
['judith', 'sutton']
['cobblestone', 'partners', 'phoenix']
['calcap', 'properties', 'inc', 'arches', 'apartment', 'homes']
['mc', 'connel', 'family', 'trust']
['tempe', 'vista']
['carlos', 'duarte']
['christine', 'miles']
['golden', 'mouse', 'enterprises']
['phoenix', '83rd', 'ave', 'apartmrnts']
['greenridge']
['david', 'brockman']
['valor', 'on', '8th']
['russ', 'breese']
['sienna', 'park']


In [629]:
# er
pairs = []
for idx, cp in enumerate(tqdm(cand_pairs)):
        
    sim = tfidf.similarity(*cp)
    if sim > 0.8:
        pairs.append((*cp, sim))
        
#     if idx < 50:
#         print(sim, '|', dedup_all_plaintiffs_list[cp[0]], '|', dedup_all_plaintiffs_list[cp[1]])
#     else:
#         break

100%|██████████| 1685978/1685978 [00:10<00:00, 154816.59it/s]


In [630]:
len(pairs), len(pairs) - len(cand_pairs)

(9622, -1676356)

In [631]:
# clustering
nx_graph = nx.Graph()
nx_graph.add_edges_from([(p[0], p[1]) for p in pairs])
plaintiff_clusters = list(nx.connected_components(nx_graph))
len(plaintiff_clusters)

1975

In [624]:
# log
for cid in random.sample(range(len(plaintiff_clusters)), 10):
    for idx, id_ in enumerate(plaintiff_clusters[cid]):
        if idx > 20:
            break
        print(dedup_all_plaintiffs_list[id_])
    print('-' * 10)

brentwood southern, llc
brentwood southern llc
----------
ryker 27th avenue, llc
ryker 27th avenue llc
ryker 27th ave llc
ryker 27th avenue
----------
calcap advisors, inc
calcap advisors inc.
calcap advisors inc
las calcap advisors inc
----------
the tides at tempe llc
tides  at tempe llc
tides  at south tempe
tides at south
tides at tempe llc
tides at south mountain tempe
tides at south tempe
tides at south tempe partners llc
----------
towers apartments llc
towers apartments, llc
----------
acapts, llc
acapts llc
----------
cjmg investments, llc
cjmg investments llc
----------
sun valley ranch
sun valley ranch apartments
----------
cws desert  ridge lp
cws desert ridge lp
cws desert ridge
----------
auxon orangewood place, llc
auxon orangewood place llc
----------


### complete output to file

In [702]:
# 1st-level id to name mapping
plaintiff_name_list = []
for idx, row in df_2020[['Plaintiff Name', 'Case Number']].dropna().iterrows():
    id_ = row['Case Number']
    name = row['Plaintiff Name']
    if not name:
        continue
    plaintiff_name_list.append((id_, name.strip()))
for idx, row in df_2020[['Plaintiff2 Name', 'Case Number']].dropna().iterrows():
    id_ = row['Case Number']
    name = row['Plaintiff2 Name']
    if not name:
        continue
    plaintiff_name_list.append((id_, name.strip()))
len(plaintiff_name_list)

39460

In [704]:
# 2nd-level id to name mapping
dedup_plaintiff_name = defaultdict(set)
for idx, n in enumerate(plaintiff_name_list):
    dedup_plaintiff_name[n[1].lower()].add(idx)

sec_id_mapping = []
dedup_plaintiff_name_list = []
for n, ids in dedup_plaintiff_name.items():
    sec_id_mapping.append(ids)
    dedup_plaintiff_name_list.append(n)
len(dedup_plaintiff_name_list)

12046

In [705]:
cleanco_terms = cleanco.prepare_terms()
cleanco_type_sources = typesources()
cleaned_plaintiffs = []
for d in dedup_plaintiff_name_list:
    company_type = cleanco.matches(d, cleanco_type_sources)
    cleaned_name = cleanco.basename(d, cleanco_terms, prefix=False, middle=False, suffix=True)
    cleaned_plaintiffs.append(cleaned_name)

In [706]:
# inverted index
plaintiff_ii = defaultdict(set)
plaintiff_tokens = []
for idx, name in enumerate(cleaned_plaintiffs):
    tokens = list(filter(lambda x: x != '' and x not in punctuation, name.split(' ')))
    plaintiff_tokens.append(tokens)
    for t in tokens:
        plaintiff_ii[t].add(idx)
len(plaintiff_ii)

9268

In [707]:
# candidate pairs
cand_pairs = set()
for _, ids in plaintiff_ii.items():
    for p in list(combinations(ids, 2)):
        cand_pairs.add(p)
len(cand_pairs)

1685993

In [708]:
# tfidf score
# from rltk.similarity import TF_IDF

tfidf = TF_IDF()

for idx, name in enumerate(cleaned_plaintiffs):
    tfidf.add_document(idx, plaintiff_tokens[idx])
    if idx < 10:
        print(plaintiff_tokens[idx])
tfidf.pre_compute(math_log=False)

['lsf9', 'master', 'participation', 'trust']
['fpa/wc', 'pinnacle', 'phase', 'i']
['gs', 'chandler']
['desert', 'jewel', 'apartments']
['marbeya']
['rpc', 'chamdler', 'apartments']
['ackk', 'enterprises']
['ashton', 'pointe', 'holdings']
['cove', 'phoenix']
['enclave', 'investors']


In [713]:
# er
pairs = []
for idx, cp in enumerate(tqdm(cand_pairs)):
        
    sim = tfidf.similarity(*cp)
    if sim > 0.9:
        pairs.append((*cp, sim))

100%|██████████| 1685993/1685993 [00:10<00:00, 164997.84it/s]


In [714]:
# clustering
nx_graph = nx.Graph()
nx_graph.add_edges_from([(p[0], p[1]) for p in pairs])
plaintiff_clusters = list(nx.connected_components(nx_graph))
len(plaintiff_clusters)

1785

In [720]:
with open('plaintiff_clusters_20210813.json', 'w') as f:
    output = []
    for c in plaintiff_clusters:
        obj = {}
        for id2 in c: # 2nd-level id
            for id1 in sec_id_mapping[id2]: # 1st-level id
                obj[plaintiff_name_list[id1][0]] = plaintiff_name_list[id1][1]
        output.append(obj)
    json.dump(output, f, indent=4)

### tfidf score probing

In [632]:
selected_tfidf = []
for tf in tfidf.tf.values():
    if 'paul' in tf and 'adams' in tf and 'palms' in tf:
        print('-' * 10)
        print('tf:', tf)
        print('idf:', tfidf.idf['paul'], tfidf.idf['adams'], tfidf.idf['palms'])
        print('tfidf:', {k: v * tfidf.idf[k] for k, v in tf.items()})
        selected_tfidf.append({k: v * tfidf.idf[k] for k, v in tf.items()})
    elif 'gregory' in tf and 'adams' in tf:
        print('-' * 10)
        print('tf:', tf)
        print('idf:', tfidf.idf['gregory'], tfidf.idf['adams'])
        print('tfidf:', {k: v * tfidf.idf[k] for k, v in tf.items()})
        selected_tfidf.append({k: v * tfidf.idf[k] for k, v in tf.items()})
    elif 'michael' in tf and 'adams' in tf:
        print('-' * 10)
        print('tf:', tf)
        print('idf:', tfidf.idf['michael'], tfidf.idf['adams'])
        print('tfidf:', {k: v * tfidf.idf[k] for k, v in tf.items()})
        selected_tfidf.append({k: v * tfidf.idf[k] for k, v in tf.items()})

----------
tf: {'paul': 0.3333333333333333, 'adams': 0.3333333333333333, 'palms': 0.3333333333333333}
idf: 481.84 1338.4444444444443 172.0857142857143
tfidf: {'paul': 160.61333333333332, 'adams': 446.1481481481481, 'palms': 57.36190476190476}
----------
tf: {'paul': 0.3333333333333333, 'adams': 0.3333333333333333, 'palms': 0.3333333333333333}
idf: 481.84 1338.4444444444443 172.0857142857143
tfidf: {'paul': 160.61333333333332, 'adams': 446.1481481481481, 'palms': 57.36190476190476}
----------
tf: {'michael': 0.5, 'adams': 0.5}
idf: 185.3230769230769 1338.4444444444443
tfidf: {'michael': 92.66153846153846, 'adams': 669.2222222222222}
----------
tf: {'gregory': 0.5, 'adams': 0.5}
idf: 926.6153846153846 1338.4444444444443
tfidf: {'gregory': 463.3076923076923, 'adams': 669.2222222222222}
----------
tf: {'gregory': 0.125, 'a': 0.125, 'adams,': 0.125, 'jordan': 0.125, 'adams': 0.125, 'and': 0.125, 'dana': 0.125, 'adamas': 0.125}
idf: 926.6153846153846 1338.4444444444443
tfidf: {'gregory': 115

In [633]:
def tf_idf_cosine_similarity(tfidf_dict1, tfidf_dict2):
    """
    Compute Cosine similarity for TF/IDF value dictionary
    
    Args:
        tfidf_dict1 (dict): TF/IDF dictionary for first record, format in ``{term1: tfidf value, ...}``
        tfidf_dict2 (dict): TF/IDF dictionary for second record, same format as tfidf_dict1.
    
    Returns:
        float:
    """
    v_x_y, v_x_2, v_y_2 = 0.0, 0.0, 0.0

    # intersection of dict1 and dict2
    # ignore the values that are not in both
    for t in tfidf_dict1.keys():
        if t in tfidf_dict2:
            v_x_y += tfidf_dict1[t] * tfidf_dict2[t]

    for t, tfidf in tfidf_dict1.items():
        v_x_2 += tfidf * tfidf
    for t, tfidf in tfidf_dict2.items():
        v_y_2 += tfidf * tfidf

    # cosine similarity
    print(v_x_y, v_x_2, v_y_2)
    return 0.0 if v_x_y == 0 else v_x_y / (math.sqrt(v_x_2) * math.sqrt(v_y_2))

In [634]:
# from rltk.similarity.tf_idf import tf_idf_cosine_similarity

for i in range(len(selected_tfidf)):
    for j in range(i+1, len(selected_tfidf)):
        print(tf_idf_cosine_similarity(selected_tfidf[i], selected_tfidf[j]), 
              list(selected_tfidf[i].keys()), list(selected_tfidf[j].keys()))

228135.20105838016 228135.20105838016 228135.20105838016
1.0000000000000002 ['paul', 'adams', 'palms'] ['paul', 'adams', 'palms']
298572.2551440329 228135.20105838016 456444.5434261085
0.9252504132213792 ['paul', 'adams', 'palms'] ['michael', 'adams']
298572.2551440329 228135.20105838016 662512.4004675286
0.7679912850316184 ['paul', 'adams', 'palms'] ['gregory', 'adams']
74643.06378600822 228135.20105838016 4754177.121846383
0.07167300731464235 ['paul', 'adams', 'palms'] ['gregory', 'a', 'adams,', 'jordan', 'adams', 'and', 'dana', 'adamas']
298572.2551440329 228135.20105838016 456444.5434261085
0.9252504132213792 ['paul', 'adams', 'palms'] ['michael', 'adams']
298572.2551440329 228135.20105838016 662512.4004675286
0.7679912850316184 ['paul', 'adams', 'palms'] ['gregory', 'adams']
74643.06378600822 228135.20105838016 4754177.121846383
0.07167300731464235 ['paul', 'adams', 'palms'] ['gregory', 'a', 'adams,', 'jordan', 'adams', 'and', 'dana', 'adamas']
447858.3827160493 456444.5434261085 

In [618]:
import math
# 446*669, 160**2+446**2+57**2, 92**2+669**2
446*669 / (math.sqrt(160**2+446**2+57**2) * math.sqrt(92**2+669**2))

0.9258127478295467